# Setup Environment Variables
Set up the environment variables for the Azure OpenAI endpoint and API key.

In [14]:
# Import the os module
import json

with open('config.json') as config_file:
    config = json.load(config_file)

    endpoint = config["AZURE_OPENAI_ENDPOINT"]
    key = config["AZURE_OPENAI_API_KEY"]
    deployment = config["AZURE_DEPLOYMENT"]
    version = config["API_VERSION"]

    print(endpoint)    

https://ai-training-dev826312122475.openai.azure.com/


# Initialize AzureOpenAI Client
Initialize the AzureOpenAI client with the endpoint, API key, and API version.

In [15]:
# Import the AzureOpenAI module
from openai import AzureOpenAI

# Initialize the AzureOpenAI client
client = AzureOpenAI(
  azure_endpoint = endpoint, 
  api_key=key,
  api_version=version
)

# Hello world test
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=deployment,
)

print(chat_completion.choices[0].message.content) 

This is a test.


# Define a simple text to extract information from

In [16]:
davidDescription = "David is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

## Define a prompt that contains the information on what to do

In [17]:
# A simple prompt to extract information from "student_description" in a JSON format.
davidPrompt = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
race

This is the body of text to extract the information from:
{davidDescription}
'''

## Call the chat api

In [18]:
# Generating response back from gpt-3.5-turbo
openai_response = client.chat.completions.create(
    messages = [{'role': 'user', 'content': davidPrompt}],
    model=deployment
)

print(openai_response.choices[0].message.content)

{
  "name": "David",
  "major": "computer science",
  "school": "Stanford University",
  "grades": 3.8,
  "race": "Asian American"
}


# Try another student

In [29]:
mikeDescription="Mike Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Mike is an active member of the university's Chess Club and the Rifle Association. He hopes to pursue a career in software engineering after graduating."

mikePrompt = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{mikeDescription}
'''

In [30]:
# Generating response back from gpt-3.5-turbo
openai_response = client.chat.completions.create(
    model = deployment,
    messages = [{'role': 'user', 'content': mikePrompt}]
)

# Loading the response as a JSON object
mike_response = json.loads(openai_response.choices[0].message.content)

print(mike_response['club'])

['Chess Club', 'Rifle Association']


# Add a simple function to extract the data from the response

In order to do so you must define metadata that will be used to extract the information from the response.

In [31]:
extract_student_info = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    }
]

# Use the function to extract the data from the response

In [32]:
students = [davidDescription,mikeDescription]
for i in students:
    response = client.chat.completions.create(
        model = deployment,
        messages = [{'role': 'user', 'content': i}],
        functions = extract_student_info,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Mike Patel', 'major': 'Computer Science', 'school': 'University of Michigan', 'grades': 3.7, 'club': 'Chess Club'}
